In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

import cartopy.crs as ccrs
import cartopy.feature as cfeature

import matplotlib.pyplot as plt

%matplotlib inline

# To avoid warning messages
import warnings
warnings.filterwarnings('ignore')

### Local functions

In [ ]:
def get_real_data_fName(year, date_yyyymmdd, data_path_real="/discover/nobackup/projects/gmao/advda/sakella/future_sst_fraci/GMAO_OPS_bin_data/data/", file_pref_real="sst_ice_", file_suff=".nc"):
    fName = data_path_real + str(year) + "/" + file_pref_real + "{}".format(date_yyyymmdd) + file_suff
    return fName

def get_real_data(fName, lat0, lon0, vName="SST"):
    real_data = xr.open_dataset(fName)[vName].sel(lat=lat0, lon=lon0, method='nearest').values
    return real_data

## Read pre-processed storm info.

In [ ]:
# Year and name of storm (in that year)
year = 2023

tc_name = 'franklin'
#tc_name = 'idalia'
#tc_name = 'lee'

ds_tc= xr.open_dataset(tc_name+ str(year)+'.nc')

start_date = str(ds_tc['time'][0].dt.strftime("%Y-%m-%d").values)
end_date = str(ds_tc['time'][-1].dt.strftime("%Y-%m-%d").values)

print("\nHurricane:\t{} originated on:\t{},\t dissipated on:\t{}.\n".format(tc_name.upper(), start_date, end_date))

## Get the real SST along the TC track.
- The TC _moves_! 
- Therefore at any time, it has a specific location. 
- At that coordinate (time, position), we know the true SST (because we are in hindcast mode).

In [ ]:
real_sst = np.zeros( (ds_tc['time'].shape[0]), dtype=np.float32)

for id in range(0, ds_tc['time'].shape[0]):
    fName_real_data= get_real_data_fName(year, str(ds_tc['time'][id].dt.strftime("%Y%m%d").values))
    real_sst[id] = get_real_data(fName_real_data, ds_tc['lat'].values[id], ds_tc['lon'].values[id])

ds_tc = ds_tc.assign(sst=(['time'],real_sst)) # add SST to dataset- make life easy! 

## Plot the SST along TC track

In [ ]:
fig, ax1 = plt.subplots(figsize=(8,6))
ax2 = ax1.twinx()

ds_tc.sst.plot.line(ax=ax1,  c='b', ls='-', label='real SST')
ds_tc.mslp.plot.line(ax=ax2, c='r', ls='-', label='MSLP')

ax1.set_ylabel('SST ($^\circ$K)',  c='b', fontsize=14, weight='bold')
ax1.tick_params(axis='y', labelcolor='b')

ax2.set_ylabel('MSLP (hPa)', c='r', fontsize=14,weight='bold')
ax2.tick_params(axis='y', labelcolor='r')

ax1.set_title('{} {}'.format(tc_name.upper(),year), fontsize=15)

## Get predicted SST as the TC advances in time.
### From any given date, we know _predict_ `fcst_nDays` [=10] days into future.
### As the TC advances in time, data for _past_ days is filled with _real_ data.

In [ ]:
fcst_nDays = 10
exp_dates  = pd.date_range(str(start_date), end_date, freq='D') # Forecast from: TC start date to the end date
nfcst = exp_dates.shape[0]
predicted_sst = np.full( (nfcst, ds_tc['time'].shape[0]), np.nan, dtype=np.float32)

for ifcst in range(1, nfcst+1): # each forecast
    # Each forecast is [fcst_nDays] = 10-days long
    fcst_dates = pd.date_range(start=exp_dates[0]+pd.DateOffset(days=ifcst-1), periods=fcst_nDays)
    fcst_start_date = fcst_dates[0].strftime("%Y%m%d")
    print("Forecast [{}] start date: {}".format(ifcst,fcst_start_date))
    
    index_start = np.where(ds_tc.time > fcst_dates[0])[0][0]
    past_indx   = np.arange(0, index_start) # --> real SST
    future_indx = np.arange(index_start, ds_tc['time'].shape[0]+1) # --> predicted SST
    
    #print("PAST", past_indx) # --> We know it.
    for id in range(0, past_indx.shape[0]):
        predicted_sst[ifcst-1, id] = real_sst[id]

    #print("FUTURE", future_indx[0], future_indx[-1]) # --> Unknown, predict SST.    
    fName_real_data= get_real_data_fName(year, str(ds_tc.time[future_indx[0]].dt.strftime("%Y%m%d").values))
    for id in range(future_indx[0], future_indx[-1]):
        predicted_sst[ifcst-1, id] = get_real_data(fName_real_data, ds_tc['lat'].values[id], ds_tc['lon'].values[id])

## Plot real and predicted data

In [ ]:
plt.figure( figsize=(16,8))

ds_tc.sst.plot.line(ls='-', lw=4, _labels=False, add_legend=True, label='REAL')
for ifcst in range(0, nfcst):
    plt.plot_date(ds_tc.time, predicted_sst[ifcst,:], ls='-', marker=None, label='FCST {}'.format(str(ifcst).zfill(2)))
plt.legend(ncol=5)
plt.ylabel('SST [$^\circ$K]', fontsize=12)